In [1]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [3]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

In [4]:
# ==============
# getting trainning data
# ==============

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in cell average)
def gen_pair_cell_ave(u, x, t, delta_x, delta_y, length=3, num=1000):
    pairs = []
    for i in range(num):
        t_index = random.randint(0, len(t)-2)
        x_s = random.randint(0, len(x)-2*length-3)
        x_e = x_s+2*length+2
        starting = x[x_s]
        ending = x[x_e]
        input_data = []
        for i in range(x_s, x_e-1):
            value = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[0]), x[i], x[i+1])
            value = value[0] * (1/delta_x)
            input_data.append(value)
        target_data = integrate.quad(lambda x: heat_equ_analytical_solu(x, t[t_index+1]), x[x_s+length], x[x_s+length+1])
        target_data = target_data[0] * (1/delta_x)
        pair = {'input': input_data, 'solu': target_data}
        pairs.append(pair)
    return pairs

In [5]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h1, h2, o, twolayers):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h1)
        self.relu1 = nn.ReLU()
        self.linear21 = nn.Linear(h1, h2)
        self.relu21 = nn.ReLU()
        self.linear22 = nn.Linear(h2, o)
        self.relu22 = nn.ReLU()
        self.linear23 = nn.Linear(h1, o)
        self.relu23 = nn.ReLU()
        self.twolayers = twolayers
        
    def forward(self, x):
        if self.twolayers == True:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear21(out)
            out = self.relu21(out)
            out = self.linear22(out)
            out = self.relu22(out)
        else:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear23(out)
            out = self.relu23(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [6]:
def gen_pair_by_xt(delta_x=1/20, delta_y=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, length=3, num=10000, code=2):
    print("====================================================")
    print("Generating pairs, the number is", num, "; the cell average list length is", 2*length+1)
    print("delta x is equal to:", delta_x, ";delta t is equal to:", delta_y)
    print("====================================================")
    print("Generating space and time segements...")
    x = arange(xmin, xmax, delta_x)
    t = arange(ymin, ymax, delta_y)
    print("Generating meshes...")
    X,T = meshgrid(x, t) # grid of point
    print("Generating analytical solution...")
    Z = heat_equ_analytical_solu(X, T) # evaluation of the function on the grid
    print("Generating pairs compelete...")
    if code == 0:
        return gen_pair(Z, len(Z[0]), len(Z), length=length, num=num)
    if code == 1:
        return gen_pair_ave(Z, len(Z[0]), len(Z), length=length, num=num)
    if code == 2:
        return gen_pair_cell_ave(Z, x, t, delta_x=1/20, delta_y=1/20, length=length, num=num)
    else:
        return gen_pair(Z, len(Z[0]), len(Z), length=length, num=num)

In [7]:
def trainning_the_model(data_pairs, hidden=6, iteration=2, twolayers=False, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False, pair_file='list_of_pairs_11 xx.txt', loss_file='list_of_loss_11 xx.txt', output_file='list_of_output_11 xx.txt', model_name="model_xx delta x=1 20 delta t=1 20"):
    print("====================================================")
    print("Neuron number in each hidden layer is:", hidden)
    print("In total we will train the data:", iteration)
    print("Is there two layers?", twolayers)
    print("Learning rate is:", lr)
    print("Learning rate decay is:", weight_decay)
    print("Saving input to:", pair_file)
    print("Saving losses to:", loss_file)
    print("Saving prediction to:", output_file)
    print("Saving model to:", model_name)
    print("====================================================")
    print("Saving the input data...")
    # save the pairs
    list_of_pair_file = open(pair_file, 'w+')
    for value in data_pairs:
        list_of_pair_file.write(str(value)+" ")
    list_of_pair_file.close()
    print("Preparing the model and the optimizer...")
    # setting up the model
    model = ResNet(7, hidden, hidden, 1, twolayers=twolayers)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad)
    criterion = nn.MSELoss()
    model.train()
    list_of_loss = []
    list_of_output = []
    print("Starting training...")
    # train for each data pair
    # by interation=2 times
    for data in data_pairs:
        for itera in range(iteration):
            output = model(torch.FloatTensor(data["input"]))
            loss = criterion(output, torch.FloatTensor([data["solu"]]))
            list_of_loss.append(np.log10(loss.item()))
            list_of_output.append(output)
            model.zero_grad()
            loss.backward()
            optimizer.step()
    print("Training complete...")
    print("Saving the training losses...")
    # save the list_of_loss
    list_of_loss_file = open(loss_file, 'w+')
    for value in list_of_loss:
        list_of_loss_file.write(str(value)+" ")
    list_of_loss_file.close()
    print("Saving the output/prediction data...")
    # save the list_of_output
    list_of_output_file = open(output_file, 'w+')
    for value in list_of_output:
        list_of_output_file.write(str(value)+" ")
    list_of_output_file.close()
    print("Saving the model itslef...")
    # finally, save the model
    model.save_model(model_name)

In [8]:
pairs_20_20 = gen_pair_by_xt(delta_x=1/20, delta_y=1/20)
pairs_20_40 = gen_pair_by_xt(delta_x=1/20, delta_y=1/40)
pairs_20_80 = gen_pair_by_xt(delta_x=1/20, delta_y=1/80)
pairs_20_160 = gen_pair_by_xt(delta_x=1/20, delta_y=1/160)
pairs_40_20 = gen_pair_by_xt(delta_x=1/40, delta_y=1/20)
pairs_40_40 = gen_pair_by_xt(delta_x=1/40, delta_y=1/40)
pairs_40_80 = gen_pair_by_xt(delta_x=1/40, delta_y=1/80)
pairs_40_160 = gen_pair_by_xt(delta_x=1/40, delta_y=1/160)
pairs_80_20 = gen_pair_by_xt(delta_x=1/80, delta_y=1/20)
pairs_80_40 = gen_pair_by_xt(delta_x=1/80, delta_y=1/40)
pairs_80_80 = gen_pair_by_xt(delta_x=1/80, delta_y=1/80)
pairs_80_160 = gen_pair_by_xt(delta_x=1/80, delta_y=1/160)
pairs_160_20 = gen_pair_by_xt(delta_x=1/160, delta_y=1/20)
pairs_160_40 = gen_pair_by_xt(delta_x=1/160, delta_y=1/40)
pairs_160_80 = gen_pair_by_xt(delta_x=1/160, delta_y=1/80)
pairs_160_160 = gen_pair_by_xt(delta_x=1/160, delta_y=1/160)

Generating pairs, the number is 10000 ; the cell average list length is 7
delta x is equal to: 0.05 ;delta t is equal to: 0.05
Generating space and time segements...
Generating meshes...
Generating analytical solution...
Generating pairs compelete...
Generating pairs, the number is 10000 ; the cell average list length is 7
delta x is equal to: 0.05 ;delta t is equal to: 0.025
Generating space and time segements...
Generating meshes...
Generating analytical solution...
Generating pairs compelete...
Generating pairs, the number is 10000 ; the cell average list length is 7
delta x is equal to: 0.05 ;delta t is equal to: 0.0125
Generating space and time segements...
Generating meshes...
Generating analytical solution...
Generating pairs compelete...
Generating pairs, the number is 10000 ; the cell average list length is 7
delta x is equal to: 0.05 ;delta t is equal to: 0.00625
Generating space and time segements...
Generating meshes...
Generating analytical solution...
Generating pairs com

# Control group

In [9]:
trainning_the_model(pairs_20_20, iteration=1, twolayers=False, pair_file='./input/pairs pairs_20_20 iteration=1 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_20 iteration=1 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_20 iteration=1 layers=1 xx.txt', model_name="./model/model pairs_20_20 iteration=1 layers=1 xx")
trainning_the_model(pairs_20_40, iteration=1, twolayers=False, pair_file='./input/pairs pairs_20_40 iteration=1 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_40 iteration=1 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_40 iteration=1 layers=1 xx.txt', model_name="./model/model pairs_20_40 iteration=1 layers=1 xx")
trainning_the_model(pairs_20_80, iteration=1, twolayers=False, pair_file='./input/pairs pairs_20_80 iteration=1 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_80 iteration=1 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_80 iteration=1 layers=1 xx.txt', model_name="./model/model pairs_20_80 iteration=1 layers=1 xx")
trainning_the_model(pairs_20_160, iteration=1, twolayers=False, pair_file='./input/pairs pairs_20_160 iteration=1 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_160 iteration=1 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_160 iteration=1 layers=1 xx.txt', model_name="./model/model pairs_20_160 iteration=1 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_20 iteration=1 layers=1 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_20 iteration=1 layers=1 xx.txt
Saving prediction to: ./result/list_of_output pairs_20_20 iteration=1 layers=1 xx.txt
Saving model to: ./model/model pairs_20_20 iteration=1 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_40 iteration=1 layers=1 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_40 iteration=1 layers=1 xx.txt
Saving prediction to: ./result/list_o

In [10]:
trainning_the_model(pairs_40_20, iteration=1, twolayers=False, pair_file='./input/pairs pairs_40_20 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_20 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_40_20 iteration=1 layers=1 xx.txt', model_name="model/model pairs_40_20 iteration=1 layers=1 xx")
trainning_the_model(pairs_40_40, iteration=1, twolayers=False, pair_file='./input/pairs pairs_40_40 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_40 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_40_40 iteration=1 layers=1 xx.txt', model_name="model/model pairs_40_40 iteration=1 layers=1 xx")
trainning_the_model(pairs_40_80, iteration=1, twolayers=False, pair_file='./input/pairs pairs_40_80 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_80 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_40_80 iteration=1 layers=1 xx.txt', model_name="model/model pairs_40_80 iteration=1 layers=1 xx")
trainning_the_model(pairs_40_160, iteration=1, twolayers=False, pair_file='./input/pairs pairs_40_160 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_160 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_40_160 iteration=1 layers=1 xx.txt', model_name="model/model pairs_40_160 iteration=1 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_20 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_40_20 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_40_20 iteration=1 layers=1 xx.txt
Saving model to: model/model pairs_40_20 iteration=1 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_40 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_40_40 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_output p

In [11]:
trainning_the_model(pairs_80_20, iteration=1, twolayers=False, pair_file='./input/pairs pairs_80_20 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_20 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_80_20 iteration=1 layers=1 xx.txt', model_name="model/model pairs_80_20 iteration=1 layers=1 xx")
trainning_the_model(pairs_80_40, iteration=1, twolayers=False, pair_file='./input/pairs pairs_80_40 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_40 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_80_40 iteration=1 layers=1 xx.txt', model_name="model/model pairs_80_40 iteration=1 layers=1 xx")
trainning_the_model(pairs_80_80, iteration=1, twolayers=False, pair_file='./input/pairs pairs_80_80 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_80 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_80_80 iteration=1 layers=1 xx.txt', model_name="model/model pairs_80_80 iteration=1 layers=1 xx")
trainning_the_model(pairs_80_160, iteration=1, twolayers=False, pair_file='./input/pairs pairs_80_160 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_160 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_80_160 iteration=1 layers=1 xx.txt', model_name="model/model pairs_80_160 iteration=1 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_20 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_80_20 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_80_20 iteration=1 layers=1 xx.txt
Saving model to: model/model pairs_80_20 iteration=1 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_40 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_80_40 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_output p

In [12]:
trainning_the_model(pairs_160_20, iteration=1, twolayers=False, pair_file='./input/pairs pairs_160_20 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_20 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_160_20 iteration=1 layers=1 xx.txt', model_name="model/model pairs_160_20 iteration=1 layers=1 xx")
trainning_the_model(pairs_160_40, iteration=1, twolayers=False, pair_file='./input/pairs pairs_160_40 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_40 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_160_40 iteration=1 layers=1 xx.txt', model_name="model/model pairs_160_40 iteration=1 layers=1 xx")
trainning_the_model(pairs_160_80, iteration=1, twolayers=False, pair_file='./input/pairs pairs_160_80 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_80 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_160_80 iteration=1 layers=1 xx.txt', model_name="model/model pairs_160_80 iteration=1 layers=1 xx")
trainning_the_model(pairs_160_160, iteration=1, twolayers=False, pair_file='./input/pairs pairs_160_160 iteration=1 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_160 iteration=1 layers=1 xx.txt', output_file='result/list_of_output pairs_160_160 iteration=1 layers=1 xx.txt', model_name="model/model pairs_160_160 iteration=1 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_20 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_160_20 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_160_20 iteration=1 layers=1 xx.txt
Saving model to: model/model pairs_160_20 iteration=1 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_40 iteration=1 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_160_40 iteration=1 layers=1 xx.txt
Saving prediction to: result/list_of_ou

# Add layer

In [13]:
trainning_the_model(pairs_20_20, iteration=1, twolayers=True, pair_file='./input/pairs pairs_20_20 iteration=1 layers=2 xx.txt', loss_file='./result/list_of_loss pairs_20_20 iteration=1 layers=2 xx.txt', output_file='./result/list_of_output pairs_20_20 iteration=1 layers=2 xx.txt', model_name="./model/model pairs_20_20 iteration=1 layers=2 xx")
trainning_the_model(pairs_20_40, iteration=1, twolayers=True, pair_file='./input/pairs pairs_20_40 iteration=1 layers=2 xx.txt', loss_file='./result/list_of_loss pairs_20_40 iteration=1 layers=2 xx.txt', output_file='./result/list_of_output pairs_20_40 iteration=1 layers=2 xx.txt', model_name="./model/model pairs_20_40 iteration=1 layers=2 xx")
trainning_the_model(pairs_20_80, iteration=1, twolayers=True, pair_file='./input/pairs pairs_20_80 iteration=1 layers=2 xx.txt', loss_file='./result/list_of_loss pairs_20_80 iteration=1 layers=2 xx.txt', output_file='./result/list_of_output pairs_20_80 iteration=1 layers=2 xx.txt', model_name="./model/model pairs_20_80 iteration=1 layers=2 xx")
trainning_the_model(pairs_20_160, iteration=1, twolayers=True, pair_file='./input/pairs pairs_20_160 iteration=1 layers=2 xx.txt', loss_file='./result/list_of_loss pairs_20_160 iteration=1 layers=2 xx.txt', output_file='./result/list_of_output pairs_20_160 iteration=1 layers=2 xx.txt', model_name="./model/model pairs_20_160 iteration=1 layers=2 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_20 iteration=1 layers=2 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_20 iteration=1 layers=2 xx.txt
Saving prediction to: ./result/list_of_output pairs_20_20 iteration=1 layers=2 xx.txt
Saving model to: ./model/model pairs_20_20 iteration=1 layers=2 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_40 iteration=1 layers=2 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_40 iteration=1 layers=2 xx.txt
Saving prediction to: ./result/list_of_

In [14]:
trainning_the_model(pairs_40_20, iteration=1, twolayers=True, pair_file='./input/pairs pairs_40_20 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_40_20 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_40_20 iteration=1 layers=2 xx.txt', model_name="model/model pairs_40_20 iteration=1 layers=2 xx")
trainning_the_model(pairs_40_40, iteration=1, twolayers=True, pair_file='./input/pairs pairs_40_40 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_40_40 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_40_40 iteration=1 layers=2 xx.txt', model_name="model/model pairs_40_40 iteration=1 layers=2 xx")
trainning_the_model(pairs_40_80, iteration=1, twolayers=True, pair_file='./input/pairs pairs_40_80 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_40_80 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_40_80 iteration=1 layers=2 xx.txt', model_name="model/model pairs_40_80 iteration=1 layers=2 xx")
trainning_the_model(pairs_40_160, iteration=1, twolayers=True, pair_file='./input/pairs pairs_40_160 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_40_160 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_40_160 iteration=1 layers=2 xx.txt', model_name="model/model pairs_40_160 iteration=1 layers=2 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_20 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_40_20 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_output pairs_40_20 iteration=1 layers=2 xx.txt
Saving model to: model/model pairs_40_20 iteration=1 layers=2 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_40 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_40_40 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_output pai

In [15]:
trainning_the_model(pairs_80_20, iteration=1, twolayers=True, pair_file='./input/pairs pairs_80_20 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_80_20 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_80_20 iteration=1 layers=2 xx.txt', model_name="model/model pairs_80_20 iteration=1 layers=2 xx")
trainning_the_model(pairs_80_40, iteration=1, twolayers=True, pair_file='./input/pairs pairs_80_40 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_80_40 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_80_40 iteration=1 layers=2 xx.txt', model_name="model/model pairs_80_40 iteration=1 layers=2 xx")
trainning_the_model(pairs_80_80, iteration=1, twolayers=True, pair_file='./input/pairs pairs_80_80 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_80_80 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_80_80 iteration=1 layers=2 xx.txt', model_name="model/model pairs_80_80 iteration=1 layers=2 xx")
trainning_the_model(pairs_80_160, iteration=1, twolayers=True, pair_file='./input/pairs pairs_80_160 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_80_160 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_80_160 iteration=1 layers=2 xx.txt', model_name="model/model pairs_80_160 iteration=1 layers=2 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_20 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_80_20 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_output pairs_80_20 iteration=1 layers=2 xx.txt
Saving model to: model/model pairs_80_20 iteration=1 layers=2 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_40 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_80_40 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_output pai

In [16]:
trainning_the_model(pairs_160_20, iteration=1, twolayers=True, pair_file='./input/pairs pairs_160_20 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_160_20 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_160_20 iteration=1 layers=2 xx.txt', model_name="model/model pairs_160_20 iteration=1 layers=2 xx")
trainning_the_model(pairs_160_40, iteration=1, twolayers=True, pair_file='./input/pairs pairs_160_40 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_160_40 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_160_40 iteration=1 layers=2 xx.txt', model_name="model/model pairs_160_40 iteration=1 layers=2 xx")
trainning_the_model(pairs_160_80, iteration=1, twolayers=True, pair_file='./input/pairs pairs_160_80 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_160_80 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_160_80 iteration=1 layers=2 xx.txt', model_name="model/model pairs_160_80 iteration=1 layers=2 xx")
trainning_the_model(pairs_160_160, iteration=1, twolayers=True, pair_file='./input/pairs pairs_160_160 iteration=1 layers=2 xx.txt', loss_file='result/list_of_loss pairs_160_160 iteration=1 layers=2 xx.txt', output_file='result/list_of_output pairs_160_160 iteration=1 layers=2 xx.txt', model_name="model/model pairs_160_160 iteration=1 layers=2 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_20 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_160_20 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_output pairs_160_20 iteration=1 layers=2 xx.txt
Saving model to: model/model pairs_160_20 iteration=1 layers=2 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 1
Is there two layers? True
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_40 iteration=1 layers=2 xx.txt
Saving losses to: result/list_of_loss pairs_160_40 iteration=1 layers=2 xx.txt
Saving prediction to: result/list_of_outp

# Add iteration

In [17]:
trainning_the_model(pairs_20_20, iteration=2, twolayers=False, pair_file='./input/pairs pairs_20_20 iteration=2 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_20 iteration=2 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_20 iteration=2 layers=1 xx.txt', model_name="./model/model pairs_20_20 iteration=2 layers=1 xx")
trainning_the_model(pairs_20_40, iteration=2, twolayers=False, pair_file='./input/pairs pairs_20_40 iteration=2 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_40 iteration=2 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_40 iteration=2 layers=1 xx.txt', model_name="./model/model pairs_20_40 iteration=2 layers=1 xx")
trainning_the_model(pairs_20_80, iteration=2, twolayers=False, pair_file='./input/pairs pairs_20_80 iteration=2 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_80 iteration=2 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_80 iteration=2 layers=1 xx.txt', model_name="./model/model pairs_20_80 iteration=2 layers=1 xx")
trainning_the_model(pairs_20_160, iteration=2, twolayers=False, pair_file='./input/pairs pairs_20_160 iteration=2 layers=1 xx.txt', loss_file='./result/list_of_loss pairs_20_160 iteration=2 layers=1 xx.txt', output_file='./result/list_of_output pairs_20_160 iteration=2 layers=1 xx.txt', model_name="./model/model pairs_20_160 iteration=2 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_20 iteration=2 layers=1 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_20 iteration=2 layers=1 xx.txt
Saving prediction to: ./result/list_of_output pairs_20_20 iteration=2 layers=1 xx.txt
Saving model to: ./model/model pairs_20_20 iteration=2 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_20_40 iteration=2 layers=1 xx.txt
Saving losses to: ./result/list_of_loss pairs_20_40 iteration=2 layers=1 xx.txt
Saving prediction to: ./result/list_o

In [18]:
trainning_the_model(pairs_40_20, iteration=2, twolayers=False, pair_file='./input/pairs pairs_40_20 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_20 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_40_20 iteration=2 layers=1 xx.txt', model_name="model/model pairs_40_20 iteration=2 layers=1 xx")
trainning_the_model(pairs_40_40, iteration=2, twolayers=False, pair_file='./input/pairs pairs_40_40 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_40 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_40_40 iteration=2 layers=1 xx.txt', model_name="model/model pairs_40_40 iteration=2 layers=1 xx")
trainning_the_model(pairs_40_80, iteration=2, twolayers=False, pair_file='./input/pairs pairs_40_80 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_80 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_40_80 iteration=2 layers=1 xx.txt', model_name="model/model pairs_40_80 iteration=2 layers=1 xx")
trainning_the_model(pairs_40_160, iteration=2, twolayers=False, pair_file='./input/pairs pairs_40_160 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_40_160 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_40_160 iteration=2 layers=1 xx.txt', model_name="model/model pairs_40_160 iteration=2 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_20 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_40_20 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_40_20 iteration=2 layers=1 xx.txt
Saving model to: model/model pairs_40_20 iteration=2 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_40_40 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_40_40 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_output p

In [19]:
trainning_the_model(pairs_80_20, iteration=2, twolayers=False, pair_file='./input/pairs pairs_80_20 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_20 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_80_20 iteration=2 layers=1 xx.txt', model_name="model/model pairs_80_20 iteration=2 layers=1 xx")
trainning_the_model(pairs_80_40, iteration=2, twolayers=False, pair_file='./input/pairs pairs_80_40 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_40 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_80_40 iteration=2 layers=1 xx.txt', model_name="model/model pairs_80_40 iteration=2 layers=1 xx")
trainning_the_model(pairs_80_80, iteration=2, twolayers=False, pair_file='./input/pairs pairs_80_80 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_80 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_80_80 iteration=2 layers=1 xx.txt', model_name="model/model pairs_80_80 iteration=2 layers=1 xx")
trainning_the_model(pairs_80_160, iteration=2, twolayers=False, pair_file='./input/pairs pairs_80_160 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_80_160 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_80_160 iteration=2 layers=1 xx.txt', model_name="model/model pairs_80_160 iteration=2 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_20 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_80_20 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_80_20 iteration=2 layers=1 xx.txt
Saving model to: model/model pairs_80_20 iteration=2 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_80_40 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_80_40 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_output p

In [20]:
trainning_the_model(pairs_160_20, iteration=2, twolayers=False, pair_file='./input/pairs pairs_160_20 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_20 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_160_20 iteration=2 layers=1 xx.txt', model_name="model/model pairs_160_20 iteration=2 layers=1 xx")
trainning_the_model(pairs_160_40, iteration=2, twolayers=False, pair_file='./input/pairs pairs_160_40 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_40 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_160_40 iteration=2 layers=1 xx.txt', model_name="model/model pairs_160_40 iteration=2 layers=1 xx")
trainning_the_model(pairs_160_80, iteration=2, twolayers=False, pair_file='./input/pairs pairs_160_80 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_80 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_160_80 iteration=2 layers=1 xx.txt', model_name="model/model pairs_160_80 iteration=2 layers=1 xx")
trainning_the_model(pairs_160_160, iteration=2, twolayers=False, pair_file='./input/pairs pairs_160_160 iteration=2 layers=1 xx.txt', loss_file='result/list_of_loss pairs_160_160 iteration=2 layers=1 xx.txt', output_file='result/list_of_output pairs_160_160 iteration=2 layers=1 xx.txt', model_name="model/model pairs_160_160 iteration=2 layers=1 xx")

Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_20 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_160_20 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_output pairs_160_20 iteration=2 layers=1 xx.txt
Saving model to: model/model pairs_160_20 iteration=2 layers=1 xx
Saving the input data...
Preparing the model and the optimizer...
Starting training...
Training complete...
Saving the training losses...
Saving the output/prediction data...
Saving the model itslef...
Neuron number in each hidden layer is: 6
In total we will train the data: 2
Is there two layers? False
Learning rate is: 0.001
Learning rate decay is: 0
Saving input to: ./input/pairs pairs_160_40 iteration=2 layers=1 xx.txt
Saving losses to: result/list_of_loss pairs_160_40 iteration=2 layers=1 xx.txt
Saving prediction to: result/list_of_ou